In [1]:
import re
import time
import pandas

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.support.ui import Select

In [3]:
options = Options()
options.add_experimental_option("detach", True)
options.add_argument('start-maximized')

# 사람인척하기위해서 개발자도구 network에서 user-agent추가, 언어 추가
options.add_argument('Chrome/134.0.0.0')
options.add_argument('lang=ko_KR')

# gpu 안쓰기
options.add_argument('--disable-gpu') 

In [28]:
driver = webdriver.Chrome(
service=Service(ChromeDriverManager().install()),
options=options
)

driver.get(f'https://www.koreabaseball.com/Schedule/Schedule.aspx')

In [29]:
month_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "#ddlMonth"))
        )
month_select = Select(month_btn)
month_select.select_by_value('06')

In [30]:
table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#tblScheduleList"))
        )
table_html = table.get_attribute('outerHTML')

In [7]:
table_html

'<table id="tblScheduleList" class="tbl" summary="경기일정 및 결과 표 (날짜, 시간, 경기, 게임센터, 하이라이트, TV, 라디오, 구장, 비고)">\n\t\t\t    <caption>경기일정 및 결과 표</caption>\n\t\t\t    <colgroup>\n\t\t\t\t    <col width="10%">\n\t\t\t\t    <col width="7%">\n\t\t\t\t    <col width="25%">\n\t\t\t\t    <col width="12%">\n\t\t\t\t    <col width="12%">\n\t\t\t\t    <col width="7%">\n\t\t\t\t    <col width="8%">\n\t\t\t\t    <col width="7%">\n\t\t\t\t    <col width="12%">\n\t\t\t    </colgroup>\n\t\t\t    <thead>\n\t\t\t\t    <tr>\n\t\t\t\t\t    <th>날짜</th>\n\t\t\t\t\t    <th>시간</th>\n\t\t\t\t\t    <th>경기</th>\n\t\t\t\t\t    <th>게임센터</th>\n\t\t\t\t\t    <th>하이라이트</th>\n\t\t\t\t\t    <th>TV</th>\n\t\t\t\t\t    <th>라디오</th>\n\t\t\t\t\t    <th>구장</th>\n\t\t\t\t\t    <th>비고</th>\n\t\t\t\t    </tr>\n\t\t\t    </thead>\n\t\t    <tbody><tr><td class="day" rowspan="5">03.22(토)</td><td class="time"><b>14:00</b></td><td class="play"><span>롯데</span><em><span class="lose">2</span><span>vs</span><span class="win">12</span></em><

In [31]:
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup

soup = BeautifulSoup(table_html, "html.parser")
df = pd.read_html(StringIO(str(soup)))[0]

df

,날짜,시간,경기,게임센터,하이라이트,TV,라디오,구장,비고
0,06.01(일),14:00,두산0vs1키움,리뷰,하이라이트,S-T,NaN,고척,-
1,06.01(일),17:00,삼성6vs4LG,리뷰,하이라이트,SPO-T,NaN,잠실,-
2,06.01(일),17:00,SSG4vs3롯데,리뷰,하이라이트,MS-T SS-T,NaN,사직,-
3,06.01(일),17:00,한화5vs16NC,리뷰,하이라이트,KN-T,NaN,창원,-
4,06.01(일),17:00,KIA5vs3KT,리뷰,하이라이트,SPO-2T,NaN,수원,-
5,06.03(화),14:00,LG15vs0NC,리뷰,하이라이트,SPO-T,NaN,창원,-
6,06.03(화),14:00,KT1vs10한화,리뷰,하이라이트,SS-T,NaN,대전(신),-
7,06.03(화),17:00,KIA11vs3두산,리뷰,하이라이트,KN-T,NaN,잠실,-
8,06.03(화),17:00,삼성4vs6SSG,리뷰,하이라이트,SPO-2T,NaN,문학,-
9,06.03(화),17:00,키움0vs8롯데,리뷰,하이라이트,MS-T,NaN,사직,-


In [9]:
# 필요없는 열 삭제
df.drop(['게임센터', '하이라이트', 'TV', '라디오'], axis=1, inplace=True)

# 필요없는 행 삭제
drop_row = df[df['비고']!='-'].index
df.drop(drop_row, inplace=True)
df.reset_index(drop=True, inplace=True)

In [10]:
df.head()

,날짜,시간,경기,구장,비고
0,03.22(토),14:00,롯데2vs12LG,잠실,-
1,03.22(토),14:00,두산5vs6SSG,문학,-
2,03.22(토),14:00,키움5vs13삼성,대구,-
3,03.22(토),14:00,한화4vs3KT,수원,-
4,03.22(토),14:00,NC2vs9KIA,광주,-


In [11]:
# 날짜 컬럼 분해 함수
def split_date_columns(df):
    df.insert(0, '월', df['날짜'].str[:2])
    df.insert(1, '일',df['날짜'].str[3:5])
    df.insert(2, '요일',df['날짜'].str[6])
    df.drop('날짜', axis=1, inplace=True)
    return df

In [12]:
# 경기 컬럼 정규표현식 분해 함수
# 홈팀승리 1, 무승부 0, 홈팀패배 -1
def parse_game_info(game_str):
    pattern = r'([가-힣A-Z]+)(\d+)vs(\d+)([가-힣A-Z]+)'
    match = re.match(pattern, game_str)
    if match:
        home_team, home_score, away_score, away_team = match.groups()
        home_score = int(home_score)
        away_score = int(away_score)

        # 무승부 반영
        if home_score > away_score:
            home_result = 1
        elif home_score < away_score:
            home_result = -1
        else:
            home_result = 0

        return pd.Series([home_team, home_score, away_score, away_team, home_result])
    else:
        return pd.Series([None]*5)

In [13]:
split_date_columns(df)

,월,일,요일,시간,경기,구장,비고
0,03,22,토,14:00,롯데2vs12LG,잠실,-
1,03,22,토,14:00,두산5vs6SSG,문학,-
2,03,22,토,14:00,키움5vs13삼성,대구,-
3,03,22,토,14:00,한화4vs3KT,수원,-
4,03,22,토,14:00,NC2vs9KIA,광주,-
5,03,23,일,14:00,롯데2vs10LG,잠실,-
6,03,23,일,14:00,두산2vs5SSG,문학,-
7,03,23,일,14:00,키움7vs11삼성,대구,-
8,03,23,일,14:00,한화4vs5KT,수원,-
9,03,23,일,14:00,NC5vs4KIA,광주,-


In [14]:
df[['홈팀', '홈점수', '원정점수', '원정팀', '홈팀승리']] = df['경기'].apply(parse_game_info)
df = df.drop(['경기', '비고'], axis=1)
df

,월,일,요일,시간,구장,홈팀,홈점수,원정점수,원정팀,홈팀승리
0,03,22,토,14:00,잠실,롯데,2,12,LG,-1
1,03,22,토,14:00,문학,두산,5,6,SSG,-1
2,03,22,토,14:00,대구,키움,5,13,삼성,-1
3,03,22,토,14:00,수원,한화,4,3,KT,1
4,03,22,토,14:00,광주,NC,2,9,KIA,-1
5,03,23,일,14:00,잠실,롯데,2,10,LG,-1
6,03,23,일,14:00,문학,두산,2,5,SSG,-1
7,03,23,일,14:00,대구,키움,7,11,삼성,-1
8,03,23,일,14:00,수원,한화,4,5,KT,-1
9,03,23,일,14:00,광주,NC,5,4,KIA,1


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   월       39 non-null     object
 1   일       39 non-null     object
 2   요일      39 non-null     object
 3   시간      39 non-null     object
 4   구장      39 non-null     object
 5   홈팀      39 non-null     object
 6   홈점수     39 non-null     int64 
 7   원정점수    39 non-null     int64 
 8   원정팀     39 non-null     object
 9   홈팀승리    39 non-null     int64 
dtypes: int64(3), object(7)
memory usage: 3.2+ KB


In [16]:
driver.quit()

## 코드 합치기

In [17]:
options = Options()
options.add_experimental_option("detach", True)
options.add_argument('start-maximized')

# 사람인척하기위해서 개발자도구 network에서 user-agent추가, 언어 추가
options.add_argument('Chrome/134.0.0.0')
options.add_argument('lang=ko_KR')

# gpu 안쓰기
options.add_argument('--disable-gpu') 

In [34]:
def fetch_2025_kbo_results(month):
    
    # selenium으로 KBO 공식 홈페이지 접속
    driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=options
            )
    driver.get(f'https://www.koreabaseball.com/Schedule/Schedule.aspx')
    
    # 경기 기록 table 추출
    month_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "#ddlMonth"))
            )
    month_select = Select(month_btn)
    month_select.select_by_value(str(month).zfill(2))
    table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#tblScheduleList"))
            )
    table_html = table.get_attribute('outerHTML')
    driver.quit()
    
    # HTML을 데이터프레임으로!
    soup = BeautifulSoup(table_html, "html.parser")
    df = pd.read_html(StringIO(str(soup)))[0]
    
    split_date_columns(df)
    df[['홈팀', '홈점수', '원정점수', '원정팀', '홈팀승리']] = df['경기'].apply(parse_game_info)

    # 필요없는 행 삭제
    drop_row = df[df['하이라이트'].isna()].index
    df.drop(drop_row, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    # 필요없는 열 삭제
    df.drop(['게임센터', '하이라이트', 'TV', '라디오', '경기', '비고'], axis=1, inplace=True)
    
    return df

In [19]:
pd.set_option('display.float_format', '{:.0f}'.format)

In [23]:
pd.set_option('display.max_rows',1000)

In [35]:
dfs = []

for i in range(3,7):
    df = fetch_2025_kbo_results(i)
    dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)
display(merged_df)

,월,일,요일,시간,구장,홈팀,홈점수,원정점수,원정팀,홈팀승리
0,03,22,토,14:00,잠실,롯데,2,12,LG,-1
1,03,22,토,14:00,문학,두산,5,6,SSG,-1
2,03,22,토,14:00,대구,키움,5,13,삼성,-1
3,03,22,토,14:00,수원,한화,4,3,KT,1
4,03,22,토,14:00,광주,NC,2,9,KIA,-1
5,03,23,일,14:00,잠실,롯데,2,10,LG,-1
6,03,23,일,14:00,문학,두산,2,5,SSG,-1
7,03,23,일,14:00,대구,키움,7,11,삼성,-1
8,03,23,일,14:00,수원,한화,4,5,KT,-1
9,03,23,일,14:00,광주,NC,5,4,KIA,1


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   월       39 non-null     object
 1   일       39 non-null     object
 2   요일      39 non-null     object
 3   시간      39 non-null     object
 4   구장      39 non-null     object
 5   홈팀      39 non-null     object
 6   홈점수     39 non-null     int64 
 7   원정점수    39 non-null     int64 
 8   원정팀     39 non-null     object
 9   홈팀승리    39 non-null     int64 
dtypes: int64(3), object(7)
memory usage: 3.2+ KB


In [37]:
import os
import pandas as pd

# 1. 상위 폴더 기준 경로 계산
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_dir = os.path.join(parent_dir, 'data')

# 2. data 폴더 없으면 생성
os.makedirs(data_dir, exist_ok=True)

# 3. 파일 경로 설정
file_path = os.path.join(data_dir, '2025_KBO_승패기록.csv')

# 4. 저장
merged_df.to_csv(file_path, encoding='utf-8-sig', index=False)

print(f"파일이 저장되었습니다: {file_path}")

파일이 저장되었습니다: C:\KBO-WinPredictor\data\2025_KBO_승패기록.csv
